In [1]:
#시뮬레이션을 위한 라이브러리들
#Import libraries for simulation
import tensorflow as tf
import numpy as np

#시각화를 위한 import들
#Imports for visualization
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def DisplayFractal(a, fmt='jpeg'):
  """Display an array of iteration counts as a
     colorful picture of a fractal."""
  a_cyclic = (6.28*a/20.0).reshape(list(a.shape)+[1])
  img = np.concatenate([10+20*np.cos(a_cyclic),
                        30+50*np.sin(a_cyclic),
                        155-80*np.cos(a_cyclic)], 2)
  img[a==a.max()] = 0
  a = img
  a = np.uint8(np.clip(a, 0, 255))
  plt.imshow(a, interpolation='nearest')
  ax = plt.gca()
  ax.axes.get_xaxis().set_ticks([])
  ax.axes.get_yaxis().set_ticks([])
  plt.show()

In [3]:
sess = tf.InteractiveSession()

#Use NumPy to create a 2D array of complex numbers on [-2,2]x[-2,2]
Y, X = np.mgrid[-1.3:1.3:0.005, -2:1:0.005]
Z = X+1j*Y

xs = tf.constant(Z.astype(np.complex64))
zs = tf.Variable(xs)
ns = tf.Variable(tf.zeros_like(xs, tf.float32))

tf.initialize_all_variables().run()

UnimplementedError: Cast complex64 to float is not supported
	 [[Node: zeros_like/Cast = Cast[DstT=DT_FLOAT, SrcT=DT_COMPLEX64, _device="/job:localhost/replica:0/task:0/cpu:0"](zeros_like/ZerosLike)]]
Caused by op u'zeros_like/Cast', defined at:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/JH/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/JH/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 589, in launch_instance
    app.start()
  File "/Users/JH/tensorflow/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 405, in start
    ioloop.IOLoop.instance().start()
  File "/Users/JH/tensorflow/lib/python2.7/s

In [6]:
#Compute the new values of z: z^2 + x
zs_ = zs*zs + xs

#Have we diverged with this new value?
not_diverged = tf.complex_abs(zs_) < 4

#Operation to update the zs and the iteration count.
#
#Note: We keep computing zs after they diverge! This
#is very wasteful! There are better, if a little
#less simple, ways to do this.
#
step = tf.group(
  zs.assign(zs_),
  ns.assign_add(tf.cast(not_diverged, tf.float32))
  )

NameError: name 'zs_' is not defined